# *Deep Reinforcement Learning*

# 0.Install Dependencies

In [50]:
!pip install tensorflow==2.3.0
!pip install gym 
!pip install keras
!pip install keras-rl2
!pip install pygame

<Surface(640x480x32 SW)>

# 1.Test Random Environment with OpenAl Gym

In [72]:
import random
import gym
import os
os.environ['SDL_VIDEODRIVER']='dummy'

In [73]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
print("number os states:", states)
actions = env.action_space.n
print("number os actions:", actions)

number os states: 4
number os actions: 2


/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

In [74]:
episodes = 10
for episode in range(1, episodes+10):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))

/usr/local/lib/python3.7/dist-packages/gym/core.py:50: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  "You are calling render method, "


Episode:1 Score:39.0
Episode:2 Score:19.0
Episode:3 Score:12.0
Episode:4 Score:15.0
Episode:5 Score:34.0
Episode:6 Score:14.0
Episode:7 Score:14.0
Episode:8 Score:34.0
Episode:9 Score:15.0
Episode:10 Score:15.0
Episode:11 Score:16.0
Episode:12 Score:26.0
Episode:13 Score:25.0
Episode:14 Score:11.0
Episode:15 Score:17.0
Episode:16 Score:35.0
Episode:17 Score:11.0
Episode:18 Score:9.0
Episode:19 Score:27.0


# 2.Create a Deep Learning Model with Keras

In [75]:
# import neccessary modules
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [76]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(units=24, activation='relu' ))
    model.add(Dense(units=24, activation='relu'))
    model.add(Dense(units=actions, activation='linear'))
    return model
    

In [78]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3.Build Agent with Keras-RL

In [77]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory 

In [83]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn


In [84]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 119s 12ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 16.368 - mae: 51.454 - mean_q: 103.278

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 127s 13ms/step - reward: 1.0000
67 episodes - episode_reward: 149.104 [18.000, 200.000] - loss: 10.868 - mae: 42.239 - mean_q: 84.913

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 125s 12ms/step - reward: 1.0000
104 episodes - episode_reward: 96.048 [14.000, 186.000] - loss: 16.295 - mae: 47.639 - mean_q: 95.629

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: 1.0000
55 episodes - episode_reward: 181.982 [48.000, 200.000] - loss: 8.630 - mae: 38.222 - mean_q: 76.881

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 102s 10ms/step - reward:

In [85]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))



Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [86]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


# 4.Reloading Agent From Memory

In [87]:
dqn.save_weights('weights/dqn_weights.h5f', overwrite=True)

In [88]:
del model
del dqn
del env

In [91]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])


/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

In [92]:
dqn.load_weights("weights/dqn_weights.h5f")

In [93]:
_ = dqn.test(env, nb_episodes=10, visualize=True,)

Testing for 10 episodes ...


/usr/local/lib/python3.7/dist-packages/gym/core.py:44: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
